# Basic usage

* [Definition of the pytorch module](#Definition-of-the-pytorch-module)
* [Training a classifier](#Training-a-classifier-and-making-predictions)
  * [Dataset](#A-toy-binary-classification-task)
  * [pytorch module](#Definition-of-the-pytorch-classification-module)
  * [Model training](#Defining-and-training-the-neural-net-classifier)
  * [Inference](#Making-predictions,-classification)
* [Training a regressor](#Training-a-regressor)
  * [Dataset](#A-toy-regression-task)
  * [pytorch module](#Definition-of-the-pytorch-regression-module)
  * [Model training](#Defining-and-training-the-neural-net-regressor)
  * [Inference](#Making-predictions,-regression)
* [Saving and loading a model](#Saving-and-loading-a-model)
  * [Whole model](#Saving-the-whole-model)
  * [Only parameters](#Saving-only-the-model-parameters)
* [Usage with an sklearn Pipeline](#Usage-with-an-sklearn-Pipeline)
* [Callbacks](#Callbacks)
* [Grid search](#Usage-with-sklearn-GridSearchCV)
  * [Special prefixes](#Special-prefixes)
  * [Performing a grid search](#Performing-a-grid-search)

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

torch.manual_seed(0);

## Training a classifier and making predictions

### A toy binary classification task

We load a toy classification task from `sklearn`.

In [14]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.utils import shuffle

In [15]:
X, y = make_classification(1000, 20, n_informative=10, random_state=0)
X, y = shuffle(X, y, random_state=0)
X = X.astype(np.float32)

In [16]:
X.shape, y.shape, y.mean()

((1000, 20), (1000,), 0.5)

### Definition of the `pytorch` classification `module`

We define a vanilla neural network with two hidden layers. The output layer should have 2 output units since there are two classes. In addition, it should have a softmax nonlinearity, because later, when calling `predict_proba`, the output from the `forward` call will be used.

In [17]:
class ClassifierModule(nn.Module):
    def __init__(
            self,
            num_units=10,
            nonlin=F.relu,
            dropout=0.5,
    ):
        super(ClassifierModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin
        self.dropout = dropout

        self.reset_params()
        
    def reset_params(self):
        self.dense0 = nn.Linear(20, self.num_units)
        self.nonlin = self.nonlin
        self.dropout = nn.Dropout(self.dropout)
        self.dense1 = nn.Linear(self.num_units, 10)
        self.output = nn.Linear(10, 2)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = F.relu(self.dense1(X))
        X = F.softmax(self.output(X), dim=-1)
        return X

### Defining and training the neural net classifier

We use `NeuralNetClassifier` because we're dealing with a classifcation task. The first argument should be the `pytorch module`. As additional arguments, we pass the number of epochs and the learning rate (`lr`), but those are optional.

*Note*: To use the CUDA backend, pass `device='cuda'` as an additional argument.

In [18]:
from skorch import NeuralNetClassifier

In [19]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
    batch_size=64,
)

As in `sklearn`, we call `fit` passing the input data `X` and the targets `y`. By default, `NeuralNetClassifier` makes a `StratifiedKFold` split on the data (80/20) to track the validation loss. This is shown, as well as the train loss and the accuracy on the validation set.

In [20]:
net.fit(X, y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6993       0.4700        0.6963  0.0215
      2        0.6876       0.5200        0.6855  0.0222
      3        0.6793       0.5550        0.6804  0.0220
      4        0.6707       0.5950        0.6735  0.0200
      5        0.6630       0.6650        0.6626  0.0243
      6        0.6447       0.6750        0.6495  0.0203
      7        0.6334       0.7200        0.6325  0.0225
      8        0.6301       0.7000        0.6201  0.0210
      9        0.6154       0.7100        0.6061  0.0206
     10        0.6068       0.7200        0.5946  0.0235
     11        0.5917       0.7150        0.5929  0.0210
     12        0.5844       0.7350        0.5758  0.0224
     13        0.5876       0.7100        0.5781  0.0211
     14        0.5654       0.7550        0.5596  0.0204
     15        0.5681       0.7550        0.5533  0.0245
     16        0.5496       0.7

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=2, bias=True)
  ),
)

Also, as in `sklearn`, you may call `predict` or `predict_proba` on the fitted model.

### Making predictions, classification

In [21]:
y_pred = net.predict(X[:5])
y_pred

array([0, 0, 1, 0, 1])

In [22]:
y_proba = net.predict_proba(X[:5])
y_proba

array([[0.5861401 , 0.4138599 ],
       [0.72167945, 0.2783205 ],
       [0.16547886, 0.8345211 ],
       [0.6584326 , 0.34156734],
       [0.48462617, 0.5153738 ]], dtype=float32)

## Training a regressor

### A toy regression task

In [23]:
from sklearn.datasets import make_regression

In [24]:
X_regr, y_regr = make_regression(1000, 20, n_informative=10, random_state=0)
X_regr = X_regr.astype(np.float32)
y_regr = y_regr.astype(np.float32) / 100
y_regr = y_regr.reshape(-1, 1)

In [25]:
X_regr.shape, y_regr.shape, y_regr.min(), y_regr.max()

((1000, 20), (1000, 1), -6.4901485, 6.154505)

*Note*: Regression currently requires the target to be 2-dimensional, hence the need to reshape. This should be fixed with an upcoming version of pytorch.

### Definition of the `pytorch` regression `module`

Again, define a vanilla neural network with two hidden layers. The main difference is that the output layer only has one unit and does not apply a softmax nonlinearity.

In [26]:
class RegressorModule(nn.Module):
    def __init__(
            self,
            num_units=10,
            nonlin=F.relu,
    ):
        super(RegressorModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin
        
        self.reset_params()

    def reset_params(self):
        self.dense0 = nn.Linear(20, self.num_units)
        self.nonlin = self.nonlin
        self.dense1 = nn.Linear(self.num_units, 10)
        self.output = nn.Linear(10, 1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = F.relu(self.dense1(X))
        X = self.output(X)
        return X

### Defining and training the neural net regressor

Training a regressor is almost the same as training a classifier. Mainly, we use `NeuralNetRegressor` instead of `NeuralNetClassifier` (this is the same terminology as in `sklearn`).

In [27]:
from skorch import NeuralNetRegressor

In [28]:
net_regr = NeuralNetRegressor(
    RegressorModule,
    max_epochs=20,
    lr=0.1,
)

In [29]:
net_regr.fit(X_regr, y_regr)

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        4.5682        3.6729  0.0261
      2        3.3615        1.3144  0.0225
      3        0.9085        0.5791  0.0230
      4        0.9227        0.8074  0.0192
      5        0.5005        0.1180  0.0211
      6        0.1183        0.1098  0.0224
      7        0.0984        0.0692  0.0196
      8        0.0653        0.0638  0.0236
      9        0.0531        0.0529  0.0167
     10        0.0400        0.0460  0.0212
     11        0.0318        0.0381  0.0216
     12        0.0253        0.0321  0.0166
     13        0.0207        0.0272  0.0186
     14        0.0173        0.0236  0.0219
     15        0.0147        0.0210  0.0220
     16        0.0129        0.0189  0.0178
     17        0.0115        0.0173  0.0236
     18        0.0105        0.0161  0.0169
     19        0.0097        0.0150  0.0206
     20        0.0089        0.0141  0.0244


<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=RegressorModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=1, bias=True)
  ),
)

### Making predictions, regression

You may call `predict` or `predict_proba` on the fitted model. For regressions, both methods return the same value.

In [30]:
y_pred = net_regr.predict(X_regr[:5])
y_pred

array([[ 0.6712743 ],
       [-1.5348388 ],
       [-0.7909038 ],
       [-0.23888807],
       [-0.73821217]], dtype=float32)

## Saving and loading a model

Save and load either the whole model by using pickle or just the learned model parameters by calling `save_params` and `load_params`.

### Saving the whole model

In [33]:
import pickle

In [34]:
file_name = '/tmp/mymodel.pkl'

In [35]:
with open(file_name, 'wb') as f:
    pickle.dump(net, f)

/Users/bbossan/anaconda3/envs/skorch/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type ClassifierModule. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [36]:
with open(file_name, 'rb') as f:
    new_net = pickle.load(f)

### Saving only the model parameters

This only saves and loads the proper `module` parameters, meaning that hyperparameters such as `lr` and `max_epochs` are not saved. Therefore, to load the model, we have to re-initialize it beforehand.

In [37]:
net.save_params(f_params=file_name)  # a file handler also works

In [38]:
# first initialize the model
new_net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
).initialize()

In [39]:
new_net.load_params(file_name)

## Usage within an `sklearn Pipeline`

It is possible to put the `NeuralNetClassifier` inside an `sklearn Pipeline`, as you would with any `sklearn` classifier.

In [40]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [41]:
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('net', net),
])

In [42]:
pipe.fit(X, y)

Re-initializing module.
Re-initializing optimizer.
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6959       0.5050        0.6905  0.0202
      2        0.6904       0.5200        0.6856  0.0243
      3        0.6866       0.5500        0.6812  0.0215
      4        0.6747       0.5950        0.6749  0.0202
      5        0.6734       0.6200        0.6684  0.0264
      6        0.6669       0.6250        0.6610  0.0206
      7        0.6693       0.6300        0.6539  0.0212
      8        0.6419       0.6450        0.6423  0.0249
      9        0.6467       0.6550        0.6339  0.0202
     10        0.6323       0.6750        0.6217  0.0237
     11        0.6170       0.6950        0.6092  0.0217
     12        0.6154       0.7000        0.5980  0.0202
     13        0.6159       0.7100        0.5921  0.0220
     14        0.6015       0.7100        0.5835  0.0208
     15        0.5857       0.7350   

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('net',
                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=2, bias=True)
  ),
))],
         verbose=False)

In [43]:
y_proba = pipe.predict_proba(X[:5])
y_proba

array([[0.49605048, 0.50394946],
       [0.63571143, 0.3642885 ],
       [0.14369418, 0.8563058 ],
       [0.6404413 , 0.35955864],
       [0.5492111 , 0.45078892]], dtype=float32)

To save the whole pipeline, including the `StandardScaler` and the pytorch module, use `pickle`.

## Callbacks

Adding a new callback to the model is straightforward. Below we show how to add a new callback that determines the area under the ROC (AUC) score.

In [45]:
from skorch.callbacks import EpochScoring

For our case here, since `sklearn` already implements AUC, we just pass the correct string `'roc_auc'`. We should also tell the callback that higher scores are better (to get the correct colors printed below -- by default, lower scores are assumed to be better). Furthermore, we may specify a `name` argument for `EpochScoring`, and whether to use training data (by setting `on_train=True`) or validation data (which is the default).

In [46]:
auc = EpochScoring(scoring='roc_auc', lower_is_better=False)

Finally, we pass the scoring callback to the `callbacks` parameter as a list and then call `fit`. Notice that we get the printed scores and color highlighting for free.

In [47]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
    callbacks=[auc],
)

In [48]:
net.fit(X, y)

  epoch    roc_auc    train_loss    valid_acc    valid_loss     dur
-------  ---------  ------------  -----------  ------------  ------
      1     0.4326        0.7020       0.5150        0.6973  0.0165
      2     0.4743        0.6964       0.4650        0.6941  0.0184
      3     0.5340        0.6954       0.5250        0.6914  0.0179
      4     0.5998        0.6914       0.5700        0.6890  0.0158
      5     0.6478        0.6896       0.5950        0.6858  0.0177
      6     0.6798        0.6830       0.5900        0.6821  0.0173
      7     0.7182        0.6841       0.6250        0.6779  0.0186
      8     0.7320        0.6777       0.6450        0.6727  0.0163
      9     0.7445        0.6751       0.6750        0.6668  0.0173
     10     0.7560        0.6696       0.7000        0.6579  0.0156
     11     0.7660        0.6562       0.7250        0.6475  0.0170
     12     0.7711        0.6594       0.7200        0.6396  0.0160
     13     0.7805        0.6454       0.7600   

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=2, bias=True)
  ),
)

## Usage with sklearn `GridSearchCV`

### Special prefixes

The `NeuralNet` class allows to directly access parameters of the `pytorch module` by using the `module__` prefix. So e.g. if you defined the `module` to have a `num_units` parameter, you can set it via the `module__num_units` argument. This is exactly the same logic that allows to access estimator parameters in `sklearn Pipeline`s and `FeatureUnion`s.

This feature is useful in several ways. For one, it allows to set those parameters in the model definition. Furthermore, it allows you to set parameters in an `sklearn GridSearchCV` as shown below.

In addition to the parameters prefixed by `module__`, you may access a couple of other attributes, such as those of the optimizer by using the `optimizer__` prefix (again, see below). All those special prefixes are stored in the `prefixes_` attribute:

In [49]:
print(', '.join(net.prefixes_))

module, iterator_train, iterator_valid, optimizer, criterion, callbacks, dataset


### Performing a hyperparameter search

Below we show how to perform a grid search over the learning rate (`lr`), the module's number of hidden units (`module__num_units`), the module's dropout rate (`module__dropout`), and whether the SGD optimizer should use Nesterov momentum or not (`optimizer__nesterov`).

In [58]:
from sklearn.model_selection import RandomizedSearchCV

In [59]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
    verbose=0,
    optimizer__momentum=0.9,
)

In [60]:
params = {
    'lr': [0.05, 0.1],
    'module__num_units': [10, 20],
    'module__dropout': [0, 0.5],
    'optimizer__nesterov': [False, True],
}

In [61]:
search = RandomizedSearchCV(net, params, refit=False, cv=3, scoring='accuracy', verbose=2, n_iter=8)

In [62]:
search.fit(X, y)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] optimizer__nesterov=False, module__num_units=10, module__dropout=0.5, lr=0.05 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  optimizer__nesterov=False, module__num_units=10, module__dropout=0.5, lr=0.05, total=   0.3s
[CV] optimizer__nesterov=False, module__num_units=10, module__dropout=0.5, lr=0.05 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV]  optimizer__nesterov=False, module__num_units=10, module__dropout=0.5, lr=0.05, total=   0.2s
[CV] optimizer__nesterov=False, module__num_units=10, module__dropout=0.5, lr=0.05 
[CV]  optimizer__nesterov=False, module__num_units=10, module__dropout=0.5, lr=0.05, total=   0.4s
[CV] optimizer__nesterov=True, module__num_units=20, module__dropout=0.5, lr=0.1 
[CV]  optimizer__nesterov=True, module__num_units=20, module__dropout=0.5, lr=0.1, total=   1.1s
[CV] optimizer__nesterov=True, module__num_units=20, module__dropout=0.5, lr=0.1 
[CV]  optimizer__nesterov=True, module__num_units=20, module__dropout=0.5, lr=0.1, total=   0.6s
[CV] optimizer__nesterov=True, module__num_units=20, module__dropout=0.5, lr=0.1 
[CV]  optimizer__nesterov=True, module__num_units=20, module__dropout=0.5, lr=0.1, total=   0.5s
[CV] optimizer__nesterov=False, module__num_units=20, module__dropout=0.5, lr=0.05 
[CV]  optimizer__nesterov=False, module__num_units=20, module__dropout=0.5, lr=0.05, total=   0.5

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:   13.5s finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=<class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class '__main__.ClassifierModule'>,
),
                   iid='warn', n_iter=8, n_jobs=None,
                   param_distributions={'lr': [0.05, 0.1],
                                        'module__dropout': [0, 0.5],
                                        'module__num_units': [10, 20],
                                        'optimizer__nesterov': [False, True]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=False,
                   return_train_score=False, scoring='accuracy', verbose=2)

In [63]:
print(search.best_score_, search.best_params_)

0.855 {'optimizer__nesterov': True, 'module__num_units': 20, 'module__dropout': 0, 'lr': 0.1}


Of course, we could further nest the `NeuralNetClassifier` within an `sklearn Pipeline`, in which case we just prefix the parameter by the name of the net (e.g. `net__module__num_units`).